# Imports

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd 
import os

In [3]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from datetime import datetime as dt
import pytz

/home/szelee/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
PATH = 'data/nyc-taxi/'
CSV_PATH = 'submissions/RF/'

# Helper functions

In [5]:
def split_df(df, y_fld, subset=None, random_state=42):
    
    if subset: df = df.sample(n=subset, random_state=random_state)

    y = df[y_fld].values
    df.drop(y_fld, axis=1, inplace=True)
    
    return df, y

# Load data

In [6]:
seed = 4896 # random seed for replication

In [7]:
train_df = pd.read_feather('tmp/taxi-train-clean')
test_df = pd.read_feather('tmp/taxi-test-clean')
test_df_raw = pd.read_csv(f'{PATH}test.csv', usecols=['key'])

In [8]:
train_df.shape, test_df.shape

((19503643, 43), (9914, 42))

In [9]:
X_train, y_train = split_df(train_df, 'fare_amount')

In [10]:
X_train.shape, y_train.shape, test_df.shape

((19503643, 42), (19503643,), (9914, 42))

# Batch fit

In [11]:
outdir = CSV_PATH
os.makedirs(outdir, exist_ok=True)
preds = []
n_forest=10
tz = pytz.timezone('Asia/Singapore')

In [12]:
%%time
for i in range (n_forest):
    
    new_seed = seed + i
    
    print(f"Random Forest [{i+1}/{n_forest}] of 16 trees with random seed {new_seed}")
    m = RandomForestRegressor(n_estimators=16, min_samples_leaf=3, max_features=0.5, n_jobs=-1, random_state=new_seed)
    
    print("Fitting on training data...")
    m.fit(X_train, y_train)

    print("Predicting on test data...")
    test_y_predictions = m.predict(test_df)
    preds.append(test_y_predictions)      
    
    print("Creating submission file...")
    
    cur_dt = dt.now(tz).strftime("%Y%m%d_%H%M%S")
    desc = 'Final_RF_' # description of submission for reference
    ensemb_desc = '_01x16_54M'
    subm_fn =  desc + cur_dt + ensemb_desc + '.csv'
    subm_path = f'{outdir}/{subm_fn}'

    # Write the predictions to a CSV file which we can submit to the competition.
    submission = pd.DataFrame(
        {'key': test_df_raw.key, 'fare_amount': test_y_predictions},
        columns = ['key', 'fare_amount'])
    submission.to_csv(subm_path, index = False)
    
    print(f"Submission: {subm_fn}")
    print()

Random Forest [1/10] of 16 trees with random seed 4896
Fitting on training data...
Predicting on test data...
Creating submission file...
Submission: Final_RF_20180924_220844_01x16_54M.csv

Random Forest [2/10] of 16 trees with random seed 4897
Fitting on training data...


KeyboardInterrupt: 

# Results

Seed 4896, 4897, 4898